In [ ]:
pip install -qq -U tensorflow-addons

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
NUM_CLASSES = 100
INPUT_SHAPE = (32, 32, 3)

IMAGE_SIZE = 72
PATCH_SIZE = 6
NUM_PATCHES = (IMAGE_SIZE // PATCH_SIZE) ** 2

PROJECTION_DIM = 64

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

## Data augmentation

According to DeiT, various techniqus are required to effectively train ViTs.
Thus we applied data augmentations such as CutMix, Mixup, Auto Augment, Repeated Augment to all models.

In [ ]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.Normalization(),
    tf.keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(factor=0.2),
    tf.keras.layers.RandomZoom(height_factor=0.2, width_factor=0.2),
  ],
  name='data_augmentation'
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_train)

## PatchEncoder

In [ ]:
class PatchEncoder(tf.keras.layers.Layer):
  def __init__(self, num_patches=NUM_PATCHES, projection_dim=PROJECTION_DIM, **kwargs):
    super().__init__(**kwargs)
    self.num_patches = num_patches
    self.position_embedding = tf.keras.layers.Embedding(
        input_dim=num_patches, output_dim=projection_dim
    )
    self.positions = tf.range(start=0, limit=self.num_patches, delta=1)

  def call(self, encoded_patches):
    encoded_positions = self.position_embedding(self.positions)
    encoded_patches = encoded_patches + encoded_positions
    return encoded_patches

## Shifted patch tokenization

* start with an image
* shift the image in diagonal direction
* concat the diagonally shifted images with the orignal image
* extract pathces of the concatenated images
* flatten the spatial dimension of all patches
* layer normalize the flattened patches and then project it

In [ ]:
class ShiftedPatchTokenization(layers.Layer):
  def __init__(
      self,
      image_size=IMAGE_SIZE,
      patch_size=PATCH_SIZE,
      num_patches=NUM_PATCHES,
      projection_dim=PROJECTION_DIM,
      vanilla=False,
      **kwargs
  ):
    super().__init__(**kwargs)
    self.vanilla = vanilla
    self.patch_size = patch_size
    self.num_patches = num_patches
    self.flatten_patches = layers.Reshape((num_patches, -1))
    self.layer_norm = layers.LayerNormalization(epsilon=1e-6)
    self.projection = layers.Dense(units=projection_dim)

  def crop_shift_pad(self, images, mode):
    pass
  
  def call(self, images):
    if not self.vanilla:
      # concat the shifted patches with the original image along last axis
      images = tf.concat(
          [
              images, 
              self.crop_shift_pad(images, mode='left-up'),
              self.crop_shift_pad(images, mode='left-down'),
              self.crop_shift_pad(images, mode='rigth-up'),
              self.crop_shift_pad(images, mode='right-down')
          ],
          axis=-1
      )
    # Patch generation
    patches = tf.image.extract_patches(
        images=images,
        sizes=[1, self.patch_size, self.patch_size, 1],
        strides=[1, self.patch_size, self.patch_size, 1],
        rates=[1, 1, 1, 1],
        padding='VALID'
    )
    flat_patches = self.flatten_patches(patches)
    if not self.vanilla:
      tokens = self.layer_norm(flat_patches)
      tokens = self.projection(tokens)
    else:
      tokens = self.projection(flat_patches)
    return (tokens, patches)